<a href="https://colab.research.google.com/github/sharonma1218/stats-306/blob/main/lecture09_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages(c("Lahman", "priceR", "DBI", "RSQLite", "priceR"))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proto’, ‘gsubfn’, ‘Rcpp’, ‘plogr’




In [ ]:
install.packages('nycflights13')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(tidyverse)
library(nycflights13)

# Lecture 09: More on relational data; SQL
<div style="border: 1px double black; padding: 10px; margin: 10px">

**After today's lecture you will:**
* Understand how to [filter data using joins](#Filtering-joins)
* Know to execute basic [SQL commands](#SQL-Commands)
</div>


## Review of last lecture
Last lecture we learned how to join data from one table to another. The main function we care about is `left_join(x, y, by=<key>)`, which tries to find a matching key in the `y` table for every row in the `x` table.

![match example](https://r4ds.hadley.nz/diagrams/join/left.png)

In [ ]:
# left join: keep all the observations in the left table, even if there r no matches
# left vs right join: left vs right table 
# is left_join(x,y) the same as right_join(y,x)? 

## 🤔 Quiz
Assume every flight is full. The busiest travel day in terms of total # of passengers scheduled to depart is:

<ol style="list-style-type: upper-alpha;">
    <li> Jul. 3 </li>
    <li> Nov. 26</li>
    <li>Nov. 27</li>
    <li>Dec. 26</li>
    <li>Dec. 27</li>
</ol>

(Hint: the `planes` table tells you how many passengers each airplane holds.)

In [ ]:
# total number of passengers
flights%>%
  left_join(planes,by=("tailnum"="tailnum"))%>% # combine the two tables by tailnum instead of default/year b/c these two variables in these two dfs are diff 
  select(month,day,seats)%>% # show only the variables you are interested in 
  group_by(month,day)%>% # grouping by month & day since they are the identifiers
  summarize(total=sum(seats,na.rm=TRUE))%>% # total seats for each day  
  arrange(desc(total))%>%
  print

`summarise()` has grouped output by 'month'. You can override using the
`.groups` argument.


# A tibble: 365 × 3
# Groups:   month [12]
   month   day  total
   <int> <int>  <int>
 1    11    27 118869
 2    12     2 117736
 3    11    26 117470
 4    10    18 117324
 5    10    21 117166
 6     7    26 116944
 7    11     7 116734
 8     4     3 116716
 9    11    21 116577
10    10     3 116514
# … with 355 more rows


## Filtering joins

Filtering joins allow us to filter rows on one table based on their presence or absence in another table. We've already seen some examples of this on the problem sets:

In [ ]:
top5 = count(flights, carrier) %>% top_n(5) %>% print
top5$carrier
flights_top5 = filter(flights, carrier %in% top5$carrier) %>% print

Selecting by n


# A tibble: 5 x 2
  carrier     n
  <chr>   <int>
1 AA      32729
2 B6      54635
3 DL      48110
4 EV      54173
5 UA      58665


[1] "AA" "B6" "DL" "EV" "UA"

# A tibble: 248,312 x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      517            515         2      830            819
 2  2013     1     1      533            529         4      850            830
 3  2013     1     1      542            540         2      923            850
 4  2013     1     1      544            545        -1     1004           1022
 5  2013     1     1      554            600        -6      812            837
 6  2013     1     1      554            558        -4      740            728
 7  2013     1     1      555            600        -5      913            854
 8  2013     1     1      557            600        -3      709            723
 9  2013     1     1      557            600        -3      838            846
10  2013     1     1      558            600        -2      753            745
# … with 248,302 more rows,

What if we wanted to filter based on more than one variable? For example, based on busiest days of the year:

In [ ]:
top6_days = count(flights, year, month, day) %>% top_n(6) %>% 
    arrange(n) %>% print

Selecting by n


# A tibble: 7 x 4
   year month   day     n
  <int> <int> <int> <int>
1  2013     7    18  1003
2  2013     7    25  1003
3  2013     7     8  1004
4  2013     7    10  1004
5  2013    12     2  1004
6  2013     7    11  1006
7  2013    11    27  1014


In [ ]:
# manual filter join

### Semi-join
`semi_join(x, y)` keeps all the observations in `x` that are also in `y`.
![semi join](http://r4ds.had.co.nz/diagrams/join-semi.png)

`semi_join()` takes the same `key=` options as the other join commands.

In [ ]:
x = tribble(
  ~key, ~val_x,
     1, "x1",
     2, "x2",
     3, "x3"
)
y = tribble(
  ~key, ~val_y,
     1, "y1",
     2, "y2",
     4, "y3"
)
semi_join(x,y)
left_join(x,y) # for comparison 
anti_join(x,y) # for later

Joining with `by = join_by(key)`


key,val_x
<dbl>,<chr>
1,x1
2,x2


Joining with `by = join_by(key)`


key,val_x,val_y
<dbl>,<chr>,<chr>
1,x1,y1
2,x2,y2
3,x3,NA


Joining with `by = join_by(key)`


key,val_x
<dbl>,<chr>
3,x3


In [ ]:
top5 = count(flights, carrier) %>% top_n(5) %>% print
semi_join(flights,top5)%>%print

Selecting by n


# A tibble: 5 × 2
  carrier     n
  <chr>   <int>
1 AA      32729
2 B6      54635
3 DL      48110
4 EV      54173
5 UA      58665


Joining with `by = join_by(carrier)`


# A tibble: 248,312 × 19
    year month   day dep_time sched_de…¹ dep_d…² arr_t…³ sched…⁴ arr_d…⁵ carrier
   <int> <int> <int>    <int>      <int>   <dbl>   <int>   <int>   <dbl> <chr>  
 1  2013     1     1      517        515       2     830     819      11 UA     
 2  2013     1     1      533        529       4     850     830      20 UA     
 3  2013     1     1      542        540       2     923     850      33 AA     
 4  2013     1     1      544        545      -1    1004    1022     -18 B6     
 5  2013     1     1      554        600      -6     812     837     -25 DL     
 6  2013     1     1      554        558      -4     740     728      12 UA     
 7  2013     1     1      555        600      -5     913     854      19 B6     
 8  2013     1     1      557        600      -3     709     723     -14 EV     
 9  2013     1     1      557        600      -3     838     846      -8 B6     
10  2013     1     1      558        600      -2     753     745       8 AA     
# …

### Anti-join
The opposite of a semi-join is an anti-join:
![anti-join](http://r4ds.had.co.nz/diagrams/join-anti.png)

## Filtering joins

Filtering joins allow us to filter rows on one table based on their presence or absence in another table. We've already seen some examples of this on the problem sets:

In [ ]:
dest_top6 <- count(flights, dest) %>% top_n(6)
filter(flights, dest %in% dest_top6$dest) %>% nrow

Selecting by n


[1] 94326

### Semi-join
`semi_join(x, y)` keeps all the observations in `x` that are also in `y`.
![semi join](http://r4ds.had.co.nz/diagrams/join-semi.png)

In [ ]:
# find all the flight with destinations in the top 6

### Anti-join
`anti_join(x, y)` does the opposite of `semi_join`: it *drops* all the observations in `x` that are also in `y`.
![anti join](https://r4ds.hadley.nz/diagrams/join/anti.png)

Anti-joins are useful for finding mismatches in joins. Last lecture we saw that some 50k entries in `flights` have `tailnum`s that do not exist in the `planes` table. To find out more about these flights we could use an anti-join to select only those rows:

In [ ]:
# the tailnums that are present in the flights but missing in the planes 
missing_planes = anti_join(flights, planes, by="tailnum") %>% 
                 filter(!is.na(tailnum)) %>% print

# A tibble: 50,094 × 19
    year month   day dep_time sched_de…¹ dep_d…² arr_t…³ sched…⁴ arr_d…⁵ carrier
   <int> <int> <int>    <int>      <int>   <dbl>   <int>   <int>   <dbl> <chr>  
 1  2013     1     1      558        600      -2     753     745       8 AA     
 2  2013     1     1      559        600      -1     941     910      31 AA     
 3  2013     1     1      600        600       0     837     825      12 MQ     
 4  2013     1     1      602        605      -3     821     805      16 MQ     
 5  2013     1     1      608        600       8     807     735      32 MQ     
 6  2013     1     1      611        600      11     945     931      14 UA     
 7  2013     1     1      623        610      13     920     915       5 AA     
 8  2013     1     1      624        630      -6     840     830      10 MQ     
 9  2013     1     1      628        630      -2    1137    1140      -3 AA     
10  2013     1     1      629        630      -1     824     810      14 AA     
# … 

Do we see any pattern in this table that could explain the missing tail numbers?

In [ ]:
missing_planes %>% sample_n(100) %>% print(width = Inf)

# A tibble: 100 × 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     4    14      849            855        -6     1101           1120
 2  2013     1    28     1352           1345         7     1706           1705
 3  2013     4    21     1609           1545        24     1818           1745
 4  2013     8     9     1656           1630        26     2036           1845
 5  2013     7    30     1803           1725        38     2047           2045
 6  2013     9    15     1344           1345        -1     1506           1520
 7  2013    12    12     1211           1220        -9     1349           1400
 8  2013    11    14      822            829        -7     1008           1029
 9  2013     4    18       NA            925        NA       NA           1110
10  2013     5    14      626            630        -4      748            805
   arr_delay carrier flight tai

In [ ]:
flights%>%
  filter(tailnum=="N542MQ")%>%
  print
# all the flights that were missing from the planes dataset aren't even registered on flightaware.com.
# these planes are most likely private! 

# A tibble: 363 × 19
    year month   day dep_time sched_de…¹ dep_d…² arr_t…³ sched…⁴ arr_d…⁵ carrier
   <int> <int> <int>    <int>      <int>   <dbl>   <int>   <int>   <dbl> <chr>  
 1  2013     1     1      600        600       0     837     825      12 MQ     
 2  2013     1     1     1301       1150      71    1518    1345      93 MQ     
 3  2013     1     2     1045        959      46    1242    1129      73 MQ     
 4  2013     1     3      926        920       6    1140    1125      15 MQ     
 5  2013     1     3     1445       1445       0    1726    1710      16 MQ     
 6  2013     1     5     1230       1200      30    1447    1430      17 MQ     
 7  2013     1     6     1102       1050      12    1255    1250       5 MQ     
 8  2013     1     6     1820       1830     -10    1955    2015     -20 MQ     
 9  2013     1     8     1348       1350      -2    1531    1545     -14 MQ     
10  2013     1     9      549        600     -11     815     825     -10 MQ     
# … wit

In [ ]:
planes%>%
  filter(tailnum=="N542MQ")%>%
  print
# confirms that there in the flights dataset but not in the planes 

# A tibble: 0 × 9
# … with 9 variables: tailnum <chr>, year <int>, type <chr>,
#   manufacturer <chr>, model <chr>, engines <int>, seats <int>, speed <int>,
#   engine <chr>


The one plane that is represented is N711M. You can [look it up](https://flightaware.com/resources/registration/N711MQ) in an online database and find out it's a private jet. My guess is the others are private jets too.

# SQL

SQL stands for "Structured Query Language". Many large databases are stored in SQL format, and you will probably encounter one if you work on big data and/or at a large company. 

In [ ]:
# SQLite database
library(nycflights13)
library(DBI)
library(RSQLite)
con <- dbConnect(SQLite(), ":memory:") # same as box below
copy_to(con, flights, "flights")
copy_to(con, airports, "airports")
copy_to(con, airlines, "airlines")
copy_to(con, weather, "weather")
q <- function(...) dbGetQuery(con, ...) %>% as_tibble

In [ ]:
# create ephemeral in-memory RQSLite database
con<-dbConnect(RSQLite::SQLite(),":memory:")
dbListTables(con)

character(0)

In [ ]:
dbWriteTable(con,"mpg",mpg,overwrite=T)
dbListTables(con)

[1] "mpg"

In [ ]:
# dbRemoveTable(con,"mpg") will delete the table

In [ ]:
# col names of this table:
dbListFields(con,"mpg")

[1] "manufacturer" "model"        "displ"        "year"         "cyl"         
 [6] "trans"        "drv"          "cty"          "hwy"          "fl"          
[11] "class"

In [ ]:
# reads the entire table:
dbReadTable(con,"mpg")

### Selecting data from a table
The SQL syntax for selecting column(s) from a table is
```{sql}
SELECT <col1>, <col2>, ..., <coln> FROM <table>
```
Note the similarity to the corresponding `tidyverse` command:
```{r}
select(<table>, <col1>, <col2>, ..., <coln>)
```

In [ ]:
# diff:
# SQL: separate w space, kind of like a sentence
# R: functions

In [ ]:
# let us select some records
res<-dbSendQuery(con,"SELECT*FROM mpg") # reserved set 

Warning message:
“Closing open result set, pending rows”


In [ ]:
dbFetch(res) # fetches the data frame 
# if you fetch it again, it results in an empty tibble. you'd need to put it on reserve again.

manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>


In [ ]:
res<-dbSendQuery(con,"SELECT manufacturer,model,year,cyl FROM mpg") 
# does the same as select function in R
# SQL is not case sensitive!

Warning message:
“Closing open result set, pending rows”


In [ ]:
dbFetch(res)

The special keyword `*` means "select everything" and is equivalent to `dplyr`'s `everything()`:

If you have a really big table, SQL allows you to `LIMIT` the number of rows it returns.

In [ ]:
# limit query to first 5 rows 
res<-dbSendQuery(con,"SELECT manufacturer,model,year,cyl FROM mpg limit 5") 
dbFetch(res)

Warning message:
“Closing open result set, pending rows”


manufacturer,model,year,cyl
<chr>,<chr>,<int>,<int>
audi,a4,1999,4
audi,a4,1999,4
audi,a4,2008,4
audi,a4,2008,4
audi,a4,1999,6


In [ ]:
#dbClearResult(res) - how to clear the reserved set 

### Distinct rows
Writing `SELECT DISTINCT` instead of `SELECT` will give every unique row in the data set:

### Filtering

The SQL syntax for filtering rows in a table uses the `WHERE` clause:
```{sql}
SELECT * FROM <table> WHERE dest = "IAH"
```
This is the same as:
```{r}
filter(<table>, dest == "IAH")
```
Note that SQL uses a single `=` to check equality!

### Inequality in SQL
The write that something is *not equal* in SQL, we traditionally use the `<>` operator:

In [ ]:
# all flights that are not destined for LAX

Newer dialects of SQL (including SQLite) seem to let you use `!=` as well though.

### Missing data
In SQL, missing data is coded as `NULL`. This is a special value which is analogous to `NA` in R. 

In [ ]:
# select non-null dep time: 
# q('SELECT * FROM flights WHERE dep_delay iS NOT NULL')%>% print
# same as filter(flights,!is.na(dep_delay))

Warning message:
“Closing open result set, pending rows”


ERROR: ignored

### Summarizing

The SQL syntax for summarizing is using the `GROUP BY` clause:
```{sql}
SELECT AVG(<col>) AS avg_col FROM <table> GROUP BY(<group cols>)
```
This is the same as:
```{r}
<table> %>% group_by(<group cols>) %>% summarize(avg_col = mean(<col>))
```

In [ ]:
# select average distance and count for each flight
q("SELECT year,month,day,AVG(distance) AS avg_dist,
COUNT() AS n FROM flights GROUP BY year,month,day")
# same as flights%>%group_by(year,month,day)%>%summarize(avg_dist=mean(distance),n=n())

You can find a list of the aggregation functions that SQLite supports [here](https://www.sqlite.org/lang_aggfunc.html). This is a somewhat limited set compared to richer SQL engines like MySQL.

### Joins

The SQL syntax for joins:
```{sql}
SELECT * FROM <table> LEFT JOIN <other_table> ON <left_key_col> = <right_key_col>
```
This is the same as:
```{r}
left_join(<table>, <other_table>, by = c("<left_key_col>" = "<right_key_col>"))
```

In [ ]:
# join planes by tailnum to flights 

dbGetQuery(con,
"SELECT flights.year,planes.tailnum AS flightyr, * FROM flights
LEFT JOIN planes ON flights.tailnum=planes.tailnum"
)

flights%>%
  left_join(planes,by="tailnum")%>%
  print

ERROR: ignored

Note here that SQL requires us to be explicit about which columns we are `SELECT`ing when joining multiple tables. Each column name must be prefixed with the name of the table in which it resides.

### Ranking

Ranking operates slightly differently in SQL than dplyr. The overall syntax is:


    SELECT RANK() OVER (
	    PARTITION BY <columns>
	    ORDER BY <expression1> [ASC|DESC], ...
    ) RankedCol FROM <table>
    
    
The easiest way to learn it is to see some examples.

In [ ]:
# rank based on departure delay
q('SELECT year,month,day,dep_delay,RANK () OVER (
  PARTITION BY month,day
  ORDER BY dep_delay DESC
  ) r FROM flights 
  WHERE r<=2')

flights%>%
  group_by(month,day)%>%
  mutate(r=min_rank(-dep_delay))%>%
  selecT(month,day,dep_delay,r)%>%
  filter(r<=2)

In [ ]:
# solve: 
# find the max highway miles across each manufacturer: 

manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>
audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
audi,a4,2.8,1999,6,manual(m5),f,18,26,p,compact


In [ ]:
mpg%>%head()

manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>
audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
audi,a4,2.8,1999,6,manual(m5),f,18,26,p,compact


In [ ]:
res<-dbSendQuery(con,"select manufacturer,max(hwy) from mpg group by manufacturer")
dbFetch(res)

manufacturer,max(hwy)
<chr>,<int>
audi,31
chevrolet,30
dodge,24
ford,26
honda,36
hyundai,31
jeep,22
land rover,18
lincoln,18


In [ ]:
# what about the min? 
res<-dbSendQuery(con,"select manufacturer,min(hwy) from mpg group by manufacturer")
dbFetch(res)

Warning message:
“Closing open result set, pending rows”


manufacturer,min(hwy)
<chr>,<int>
audi,23
chevrolet,14
dodge,12
ford,15
honda,29
hyundai,24
jeep,12
land rover,15
lincoln,16


In [ ]:
# quiz: 
# how many 4 cylinder models are present in this database?
res<-dbSendQuery(con,"select count() from mpg where cyl=4")
dbFetch(res)

Warning message:
“Closing open result set, pending rows”


count()
<int>
81


In [ ]:
# solve
# get all the manufacturers who have 4 or more compact car class
res<-dbSendQuery(con,"select manufacturer, 
count() from mpg 
where class='compact' 
group by manufacturer 
having count() >=4")
dbFetch(res)

Warning message:
“Closing open result set, pending rows”


manufacturer,count()
<chr>,<int>
audi,15
subaru,4
toyota,12
volkswagen,14


In [ ]:
# solve:
# distinct car manufacturer 

In [ ]:
# once done, it's a good idea to disconnect from the database
dbClearResult(res)

## Subqueries

An aspect of SQL I find confusing is that you can filter (`WHERE ...`) based on a ranking condition:

In [ ]:
# filtering a rank, won't work

Instead, you can write a *subquery*, basically stitching two SQL calls together:

    WITH (<a SQL query>) AS table
    SELECT <stuff> FROM table WHERE <conditions>
    

In [ ]:
# filtering with rank

## Advanced joins in SQL
SQL is more general in specifying the join condition. Whereas in tidyverse it must be a key, in
SQL it can be a general logical condition.

## Example
What is the closest airport to `DTW`?

In [ ]:
# find the nearest neighbor to each airport

Let's use this to build a map of every airport's nearest neighbor:

In [ ]:
# compute air_nn, every airport's nearest neighbor



In [ ]:
air_nn  %>% filter(lat < 50, lat > 23) %>%
    ggplot + 
    geom_segment(aes(x = lon, y = lat, xend = lon2, yend = lat2), color="blue") +
    geom_point(aes(x = lon, y = lat)) + 
    coord_quickmap()

ERROR: Error in filter(., lat < 50, lat > 23): object 'air_nn' not found


## Exercise
In 1985 a hurricane made landfall on Long Island, NY between JFK Airport and Islip.
What was the name of that hurricane?

In [ ]:
jfk <- airports %>% filter(faa == "JFK")
jfk$lon
jfk$lat
storms %>% mutate(dist = sqrt((lat -+ jfk$lat)^2 + (long - jfk$lon)^2)) %>% select(dist, everything()) %>%
    arrange(dist) %>% slice(1:3)

[1] -73.77893

[1] 40.63975

  dist      name   year month day hour lat  long  status              category
1 0.2817433 Floyd  1999 9     17   0   40.6 -73.5 tropical storm      0       
2 0.4805718 Gloria 1985 9     27  16   40.6 -73.3 hurricane           1       
3 0.5881268 Chris  1988 8     30   0   41.2 -73.6 tropical depression -1      
  wind pressure ts_diameter hu_diameter
1 50    980     NA          NA         
2 75    961     NA          NA         
3 20   1008     NA          NA         